In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
import IPython
import itertools
import numpy as np
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [112]:
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-blink-features=AutomationControlled')
options.headless = True

user_agent = {'User-Agent': random.choice(user_agents_list)}

options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options)

In [ ]:
def find_state(soup):
  """ Encontra o estado em que se apresenta as informações
  Retorna uma string do estado"""

  form = soup.find("div", id="form-sections")
  form_loc = form.find("select", id="location")
  state = form_loc.find("option", selected="selected").text

  return state

def find_info(soup):
  """ Encontra o parâmetro presente na tabela do site, assim como mês e ano da observação
  Retorna uma lista com 3 valores por observação ([Ano, Mês, Parâmetro])
  Obs: O parâmetro é convertido para float
  """
  info = []

  table = soup.find("tbody")
  rows = table.find_all("tr", role="row")

  for row in rows:
    date = row.find_all("td")[0].text.split()
    mth = date[0]
    year = date[1]

    data = row.find_all("td")[1].text
    data_float = float(re.findall(r"\d+.\d+", data)[0])

    info.append([year, mth, data_float])

  return info

In [113]:
len_states = 49

param = ["tavg", "pcp"]

df = pd.DataFrame()

for state_num in range(1,len_states + 1,1):

  if state_num == 49:
    state_num = 50

  for p in param:

        df_tmp = pd.DataFrame()

        time.sleep(1)

        page_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/"+str(state_num)+"/"+p+"/all/10/2009-2018"

        driver.get(page_url)
        driver.implicitly_wait(30)

        # Espera o elemento aparecer na página
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/main/div/div/div[6]/table/tbody/tr[1]"))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        state = find_state(soup)
        info = find_info(soup)

        df_tmp["Year"] = [i[0] for i in info]
        df_tmp["Month"] = [i[1] for i in info]
        df_tmp[p] = [i[2] for i in info]
        df_tmp["State"] = state

        df = pd.concat([df, df_tmp], ignore_index=True, axis=0)

df = df.groupby(['State', 'Year', 'Month']).apply(lambda group: group.ffill().bfill())

<ipython-input-113-2b7a6722c76d>:40: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby(['State', 'Year', 'Month']).apply(lambda group: group.ffill().bfill())


In [118]:
from google.colab import files

df = df[['State', 'Year', 'Month', 'tavg', 'pcp']]

df.to_csv('df_weather.csv', index=False)
files.download('df_weather.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>